In [4]:
import numpy as np
import torch
import os

In [5]:
os.chdir("C:\\Users\\Bridget Leonard\\Desktop\\DNN_fMRI")

## Part 3: Voxelwise Encoding Models

Now that we have our stimuli features from the BridgeTower model, we can use them to predict brain responses. To do this, we will need to load in our fMRI data.
To start we will be using sub1 from Popham et al 2021

### 1 Load fMRI data

In [13]:
s1_movie_test = np.load("bridgetower\\data\\moviedata\\S1\\test.npy")
s1_movie_train = np.load("bridgetower\\data\\moviedata\\S1\\train.npy")

In [ ]:
s1_alternateithicatom = np.load("bridgetower\\data\\storydata\\S1\\alternateithicatom.npy")
s1_avatar = np.load("bridgetower\\data\\storydata\\S1\\avatar.npy")
s1_howtodraw = np.load("bridgetower\\data\\storydata\\S1\\howtodraw.npy")
s1_legacy = np.load("bridgetower\\data\\storydata\\S1\\legacy.npy")
s1_life = np.load("bridgetower\\data\\storydata\\S1\\life.npy")
s1_myfirstdaywiththeyankees = np.load("bridgetower\\data\\storydata\\S1\\myfirstdaywiththeyankees.npy")
s1_naked = np.load("bridgetower\\data\\storydata\\S1\\naked.npy")
s1_odetostepfather = np.load("bridgetower\\data\\storydata\\S1\\odetostepfather.npy")
s1_souls = np.load("bridgetower\\data\\storydata\\S1\\souls.npy")
s1_undertheinfluence = np.load("bridgetower\\data\\storydata\\S1\\undertheinfluence.npy")

The data is organized as such:

`moviedata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each partcipant’s folder, there is a train.npy and a test.npy file, which contain the brain data for the training and testing data for the models. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the motion energy features (N=2139), which are then followed by the semantic features (N=985).

`storydata`: Inside this folder, there are subfolders for each participant’s data as well as one folder for the stimuli

Within each participant’s folder, there is a file for each stimulus which contains the brain data. The shape of each of these is *[n_TR, z, x, y]*, and the last three dimensions should be flattened in order to match the shape of the vertex to voxel mappings in the mappers folder. Data that is located outside of the cortical mask is indicated by np.nan values.
The shape of the data in the stimuli folder is [n_TR, n_features]. The features dimension includes the low level features (number of words (N=1), number of phonemes (N=1), and phoneme counts (N=39)), which are then followed by the semantic features (N=985).

In [15]:
print("S1 movie test shape:", s1_movie_test.shape)
print("S1 movie train shape:", s1_movie_train.shape)
# Flatten data
s1_movie_test_flat = s1_movie_test.reshape(s1_movie_test.shape[0], -1)
s1_movie_train_flat = s1_movie_train.reshape(s1_movie_train.shape[0], -1)

print("Flattened S1 movie test shape:", s1_movie_test_flat.shape)
print("Flattened S1 movie train shape:", s1_movie_train_flat.shape)

S1 movie test shape: (270, 31, 100, 100)
S1 movie train shape: (3600, 31, 100, 100)
Flattened S1 movie test shape: (270, 310000)
Flattened S1 movie train shape: (3600, 310000)


#### 1.2 Load in feature vectors

In [16]:
# movie data
test = np.load("bridgetower\\feature_vectors\\movie\\test_data.npy")
train00 = np.load("bridgetower\\feature_vectors\\movie\\train_00_data.npy")
train01 = np.load("bridgetower\\feature_vectors\\movie\\train_01_data.npy")
train02 = np.load("bridgetower\\feature_vectors\\movie\\train_02_data.npy")
train03 = np.load("bridgetower\\feature_vectors\\movie\\train_03_data.npy")
train04 = np.load("bridgetower\\feature_vectors\\movie\\train_04_data.npy")
train05 = np.load("bridgetower\\feature_vectors\\movie\\train_05_data.npy")
train06 = np.load("bridgetower\\feature_vectors\\movie\\train_06_data.npy")
train07 = np.load("bridgetower\\feature_vectors\\movie\\train_07_data.npy")
train08 = np.load("bridgetower\\feature_vectors\\movie\\train_08_data.npy")
train09 = np.load("bridgetower\\feature_vectors\\movie\\train_09_data.npy")
train10 = np.load("bridgetower\\feature_vectors\\movie\\train_10_data.npy")
train11 = np.load("bridgetower\\feature_vectors\\movie\\train_11_data.npy")

In [ ]:
# story data
alternateithicatom = np.load("bridgetower\\feature_vectors\\story\\alternateithicatom_data.npy")
avatar = np.load("bridgetower\\feature_vectors\\story\\avatar.npy")
howtodraw = np.load("bridgetower\\feature_vectors\\story\\howtodraw.npy")
legacy = np.load("bridgetower\\feature_vectors\\story\\legacy.npy")
life = np.load("bridgetower\\feature_vectors\\story\\life.npy")
myfirstdaywiththeyankees = np.load("bridgetower\\feature_vectors\\story\\myfirstdaywiththeyankees.npy")
naked = np.load("bridgetower\\feature_vectors\\story\\naked.npy")
odetostepfather = np.load("bridgetower\\feature_vectors\\story\\odetostepfather.npy")
souls = np.load("bridgetower\\feature_vectors\\story\\souls.npy")
undertheinfluence = np.load("bridgetower\\feature_vectors\\story\\undertheinfluence.npy")